In [1]:
# -*- encoding: utf-8 -*-
'''
@File    :   address_API.py
@Time    :   2020/09/07 15:57:42
@Author  :   DataMagician 
@Version :   1.0
@Contact :   408903228@qq.com
'''

# here put the import lib
import requests
import json

def getUrl(*address):
    '''
    调用地图API获取待查询地址专属url
    最高查询次数30w/天，最大并发量160/秒
    '''
    ak = 'Vhzgfjih2w3YY0WUPuPdKiMQKwIor9rp'
    if len(address) < 1:
        return None
    else:
        for add in address:   
            url = 'http://api.map.baidu.com/geocoding/v3/?address={inputAddress}&output=json&ak={myAk}'.format(inputAddress=add,myAk=ak)  
            yield url
            
def getPosition(url):
    '''返回经纬度信息'''
    res = requests.get(url)
    json_data = json.loads(res.text)
    
    if json_data['status'] == 0:
        lat = json_data['result']['location']['lat'] #纬度
        lng = json_data['result']['location']['lng'] #经度
        precise = json_data['result']['precise']
        confidence = json_data['result']['confidence']
        comprehension = json_data['result']['comprehension']
        level = json_data['result']['level']
    else:
        print("Error output!")
        return json_data['status']
    return lat,lng,precise,confidence,comprehension,level

def getStrings(address):
    for add in address:
        add_url = list(getUrl(add))[0]
        print(add_url)
        try:
            lat,lng,precise,confidence,comprehension,level = getPosition(add_url) 
            print("地址：{0}|经度:{1}|纬度:{2}|精度:{3}|置信度:{4}|解释性:{5}|分类等级:{6}".format(add,lat,lng,precise,confidence,comprehension,level))
        except Error as e:
            print(e)

def getDict(address):
    column = ["地址","经度","纬度","精度","置信度","解释性","分类等级"]
    addressL,latL,lngL,preciseL,confidenceL,comprehensionL,levelL = [],[],[],[],[],[],[]
    n = -1
    for add in address:
        addressL.append(add)
        lat,lng,precise,confidence,comprehension,level  = getPosition(list(getUrl(add))[0])
        latL.append(lat)
        lngL.append(lng)
        preciseL.append(precise)
        confidenceL.append(confidence)
        comprehensionL.append(comprehension)
        levelL.append(level)
                
    return {cn : data for cn , data in zip(column,(addressL,latL,lngL,preciseL,confidenceL,comprehensionL,levelL))}


In [2]:
str_ = "杭州灵隐寺-浙江省博物馆-浙江省图书馆-杭州晓书馆-西湖区蚂蚁金服-浙江省金华市东阳市横店镇-西湖景区-西湖青之坞-杭州小河直街"
#str_ = "杭州市武林夜市-杭州星空恋爱解压馆-杭州大厦"


In [3]:
import numpy as np
import pandas as pd

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)


def minkowski_distance(vec1, vec2, p=3):
    """
    闵氏距离
    当p=1时，就是曼哈顿距离
    当p=2时，就是欧氏距离
    当p→∞时，就是切比雪夫距离
    :param vec1:
    :param vec2:
    :param p:
    :return:
    """
    # return sum([(x - y) ** p for (x, y) in zip(vec1, vec2)]) ** (1 / p)
    return np.linalg.norm(vec1 - vec2, ord=p)

def sorted_by_distance(str_,my_p=1):
    location_info = getDict(str_.split("-"))
    df = pd.DataFrame(location_info)
    temp = np.array([[i,j,z] for i,j,z in zip(df.纬度.to_list(),df.经度.to_list(),df.地址)])
    distance = [[a[-1],b[-1],minkowski_distance(a[:-1].astype(float),b[:-1].astype(float),p=1)] for b in temp for a in temp]
    #search_index_ = {item[0]:item[-1] for item in sorted(distance.items(), key=lambda item:item[1])}
    
    #dict_ = [[a[-1],b[-1],minkowski_distance(a[:-1].astype(float),b[:-1].astype(float),p=my_p)] for b in temp for a in temp]
    table = pd.DataFrame(distance,columns=["start","end","distance"])
    newtable1 = table.sort_values(by='distance', ascending=True)
    newtable = newtable1[newtable1["start"] != newtable1["end"]]
    return newtable.reset_index(drop=True)

print("曼哈顿距离排序")
sorted_data = sorted_by_distance(str_,my_p=2)
sorted_data

曼哈顿距离排序


,start,end,distance
0,西湖区蚂蚁金服,西湖青之坞,0.022307
1,西湖青之坞,西湖区蚂蚁金服,0.022307
2,西湖景区,杭州灵隐寺,0.037297
3,杭州灵隐寺,西湖景区,0.037297
4,西湖区蚂蚁金服,杭州小河直街,0.044762
5,杭州小河直街,西湖区蚂蚁金服,0.044762
6,西湖青之坞,杭州小河直街,0.046828
7,杭州小河直街,西湖青之坞,0.046828
8,西湖区蚂蚁金服,西湖景区,0.054152
9,西湖景区,西湖区蚂蚁金服,0.054152


In [4]:
def list_info_for_each_address(strt_,p=2):
    sorted_by_distance_ = sorted_by_distance(str_,my_p=p)
    sorted_data = sorted_by_distance_.T
    dist = set(sorted_data[No]["start"] for No in sorted_data)
    for address in  dist:
        print(sorted_by_distance_[sorted_by_distance_["start"]==address])
list_info_for_each_address(str_,p=3)

     start           end  distance
5   杭州小河直街       西湖区蚂蚁金服  0.044762
7   杭州小河直街         西湖青之坞  0.046828
17  杭州小河直街          西湖景区  0.098914
18  杭州小河直街         杭州灵隐寺  0.100937
25  杭州小河直街         杭州晓书馆  0.159767
39  杭州小河直街        浙江省图书馆  0.217770
54  杭州小河直街        浙江省博物馆  0.487902
65  杭州小河直街  浙江省金华市东阳市横店镇  1.324740
    start           end  distance
3   杭州灵隐寺          西湖景区  0.037297
10  杭州灵隐寺       西湖区蚂蚁金服  0.056175
14  杭州灵隐寺         西湖青之坞  0.078042
19  杭州灵隐寺        杭州小河直街  0.100937
20  杭州灵隐寺        浙江省图书馆  0.116833
33  杭州灵隐寺         杭州晓书馆  0.193383
45  杭州灵隐寺        浙江省博物馆  0.386965
61  杭州灵隐寺  浙江省金华市东阳市横店镇  1.291124
     start           end  distance
21  浙江省图书馆         杭州灵隐寺  0.116833
22  浙江省图书馆          西湖景区  0.118856
26  浙江省图书馆         杭州晓书馆  0.172345
28  浙江省图书馆       西湖区蚂蚁金服  0.173008
35  浙江省图书馆         西湖青之坞  0.194875
38  浙江省图书馆        杭州小河直街  0.217770
43  浙江省图书馆        浙江省博物馆  0.308468
67  浙江省图书馆  浙江省金华市东阳市横店镇  1.340353
    start           end  distance
1   西湖青之坞       西湖区蚂蚁金服  0.022

In [5]:
def address_datas_url(*address_list):
    '''
    调用地图API获取待查询地址专属url
    最高查询次数30w/天，最大并发量160/秒
    '''
    ak = 'Vhzgfjih2w3YY0WUPuPdKiMQKwIor9rp'
    if len(address_list) < 1:
        return None
    else:
        result = dict()
        for address in address_list:   
            url = {address:'http://api.map.baidu.com/geocoding/v3/?address={inputAddress}&output=json&ak={myAk}'.format(inputAddress=address,myAk=ak)}  
            result.update(url)
        return result

In [1]:
#!/usr/bin/env python
"# -*- encoding: utf-8 -*-"
'''
@File  :  geographic_path_planning_test_case.py
@Author:  manman
@Date  :  2020/11/87:47 下午
@Desc  :
@File  :  
@Time  :  // ::",
@Contact :   408903228@qq.com
@Department   :  my-self
'''
import numpy as np
import pandas as pd
import requests
import json

# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)
# 设置value的显示长度为100，默认为50
pd.set_option('max_colwidth', 100)


def minkowski_distance(vec1, vec2, p=3):
    """
    闵氏距离
    当p=1时，就是曼哈顿距离
    当p=2时，就是欧氏距离
    当p→∞时，就是切比雪夫距离
    :param vec1:
    :param vec2:
    :param p:
    :return:
    """
    # return sum([(x - y) ** p for (x, y) in zip(vec1, vec2)]) ** (1 / p)
    return np.linalg.norm(vec1 - vec2, ord=p)


def address_datas_url(*address_list):
    '''
    调用地图API获取待查询地址专属url
    最高查询次数30w/天，最大并发量160/秒
    '''
    ak = 'Vhzgfjih2w3YY0WUPuPdKiMQKwIor9rp'
    if len(address_list) < 1:
        return None
    else:
        result = dict()
        for address in address_list:
            url = {
                address: 'http://api.map.baidu.com/geocoding/v3/?address={inputAddress}&output=json&ak={myAk}'.format(
                    inputAddress=address, myAk=ak)}
            result.update(url)
        return result


def geographic_info_table(address_list):
    '''
    批量转换地址
    Args:
        address_list: 地址字符串列表

    Returns:

    '''
    data_of_result = []
    for address_name in address_list:
        column_names = ["address_names", "lng", "lat", "precise", "confidence", "comprehension", "leve", "url"]
        url_dict = address_datas_url(address_name)
        url = url_dict[address_name]
        res = requests.get(url)
        json_data = json.loads(res.text)
        if json_data["status"] == 0:
            result = json_data['result']
            location = result["location"]
            lng, lat = location["lng"], location["lat"]
            precise = result["precise"]
            confidence = result["confidence"]
            comprehension = result["comprehension"]
            level = result["level"]
        else:
            result, location, lng, lat, \
            precise, confidence, comprehension, level = \
                None, None, None, None, \
                None, None, None, None
        temp = [address_name, lng, lat
            , precise, confidence, comprehension
            , level, url]
        data_of_result.append(temp)
    return pd.DataFrame(data_of_result,columns=column_names)



str_ = "杭州灵隐寺-浙江省博物馆-浙江省图书馆-杭州晓书馆-西湖区蚂蚁金服-浙江省金华市东阳市横店镇-西湖景区-西湖青之坞-杭州小河直街"
address_list = str_.split("-")
geographic_info_table_ = geographic_info_table(address_list)
geographic_info_table_

,address_names,lng,lat,precise,confidence,comprehension,leve,url
0,杭州灵隐寺,120.108260,30.246569,0,50,100,旅游景点,http://api.map.baidu.com/geocoding/v3/?address=杭州灵隐寺&output=json&ak=Vhzgfjih2w3YY0WUPuPdKiMQKwIo...
1,浙江省博物馆,119.735929,30.231935,0,50,100,旅游景点,http://api.map.baidu.com/geocoding/v3/?address=浙江省博物馆&output=json&ak=Vhzgfjih2w3YY0WUPuPdKiMQKwI...
2,浙江省图书馆,120.025229,30.212767,1,70,100,教育,http://api.map.baidu.com/geocoding/v3/?address=浙江省图书馆&output=json&ak=Vhzgfjih2w3YY0WUPuPdKiMQKwI...
3,杭州晓书馆,120.039324,30.371017,1,70,100,教育,http://api.map.baidu.com/geocoding/v3/?address=杭州晓书馆&output=json&ak=Vhzgfjih2w3YY0WUPuPdKiMQKwIo...
4,西湖区蚂蚁金服,120.131800,30.279204,1,80,100,公司企业,http://api.map.baidu.com/geocoding/v3/?address=西湖区蚂蚁金服&output=json&ak=Vhzgfjih2w3YY0WUPuPdKiMQKw...
5,浙江省金华市东阳市横店镇,120.314998,29.162183,0,25,100,乡镇,http://api.map.baidu.com/geocoding/v3/?address=浙江省金华市东阳市横店镇&output=json&ak=Vhzgfjih2w3YY0WUPuPdK...
6,西湖景区,120.127920,30.228932,0,50,100,旅游景点,http://api.map.baidu.com/geocoding/v3/?address=西湖景区&output=json&ak=Vhzgfjih2w3YY0WUPuPdKiMQKwIor9rp
7,西湖青之坞,120.153887,30.278984,0,50,100,NoClass,http://api.map.baidu.com/geocoding/v3/?address=西湖青之坞&output=json&ak=Vhzgfjih2w3YY0WUPuPdKiMQKwIo...
8,杭州小河直街,120.141920,30.313845,0,30,100,道路,http://api.map.baidu.com/geocoding/v3/?address=杭州小河直街&output=json&ak=Vhzgfjih2w3YY0WUPuPdKiMQKwI...


In [2]:
def minkowski_distance(vec1, vec2, p=3):
    """
    闵氏距离
    当p=1时，就是曼哈顿距离
    当p=2时，就是欧氏距离
    当p→∞时，就是切比雪夫距离
    :param vec1:
    :param vec2:
    :param p:
    :return:
    """
    # return sum([(x - y) ** p for (x, y) in zip(vec1, vec2)]) ** (1 / p)
    return np.linalg.norm(vec1 - vec2, ord=p)

In [3]:
def get_node(address_list):
    geographic_info_table_ = geographic_info_table(address_list)
    lnglat_pandas = geographic_info_table_[["lng","lat"]].astype(float).to_dict()
    lnglat_numpy = np.array([[l[-1] for l in lnglat_pandas[name].items()] for name in lnglat_pandas]).T
    node_dict = dict()
    for address_name1,lnglat1 in zip(geographic_info_table_.address_names,lnglat_numpy):
        for address_name2,lnglat2 in zip(geographic_info_table_.address_names,lnglat_numpy):
            from_,to_ , distance = address_name1,address_name2,minkowski_distance(lnglat1,lnglat2,p=2)
            if from_ != to_:
                if from_ in node_dict:
                    node_dict[from_].append([from_,to_,distance])
                else:
                    node_dict.update({from_:[[from_,to_,distance]]})
    return node_dict

get_node(address_list)

{'杭州灵隐寺': [['杭州灵隐寺', '浙江省博物馆', 0.37261821894154573],
  ['杭州灵隐寺', '浙江省图书馆', 0.0896475785341481],
  ['杭州灵隐寺', '杭州晓书馆', 0.14226477204155247],
  ['杭州灵隐寺', '西湖区蚂蚁金服', 0.04023898467674691],
  ['杭州灵隐寺', '浙江省金华市东阳市横店镇', 1.1039170975869144],
  ['杭州灵隐寺', '西湖景区', 0.026411843763575676],
  ['杭州灵隐寺', '西湖青之坞', 0.055969277373088766],
  ['杭州灵隐寺', '杭州小河直街', 0.07522731741274191]],
 '浙江省博物馆': [['浙江省博物馆', '杭州灵隐寺', 0.37261821894154573],
  ['浙江省博物馆', '浙江省图书馆', 0.2899342226322592],
  ['浙江省博物馆', '杭州晓书馆', 0.3337550493652799],
  ['浙江省博物馆', '西湖区蚂蚁金服', 0.3986826784442438],
  ['浙江省博物馆', '浙江省金华市东阳市横店镇', 1.2164248631900676],
  ['浙江省博物馆', '西湖景区', 0.39200247266004895],
  ['浙江省博物馆', '西湖青之坞', 0.4205975050396113],
  ['浙江省博物馆', '杭州小河直街', 0.4141717826546668]],
 '浙江省图书馆': [['浙江省图书馆', '杭州灵隐寺', 0.0896475785341481],
  ['浙江省图书馆', '浙江省博物馆', 0.2899342226322592],
  ['浙江省图书馆', '杭州晓书馆', 0.15887595383206737],
  ['浙江省图书馆', '西湖区蚂蚁金服', 0.12558339008179156],
  ['浙江省图书馆', '浙江省金华市东阳市横店镇', 1.0898129502187617],
  ['浙江省图书馆', '西湖景区', 0.10395556

In [4]:
def sorted_nodes(address_list):
    result = dict()
    for start,data in get_node(address_list).items():
        ndata=np.array(data) 
        sorted_node= ndata[np.argsort(ndata[:,-1])]
        to_flaot = sorted_node[:,-1].astype(float)
        contdata = np.c_[sorted_node[:,0:-1],to_flaot]
        result.update({start:contdata})
    return result

sorted_nodes_ = sorted_nodes(address_list)
sorted_nodes_

{'杭州灵隐寺': array([['杭州灵隐寺', '西湖景区', '0.026411843763575676'],
        ['杭州灵隐寺', '西湖区蚂蚁金服', '0.04023898467674691'],
        ['杭州灵隐寺', '西湖青之坞', '0.055969277373088766'],
        ['杭州灵隐寺', '杭州小河直街', '0.07522731741274191'],
        ['杭州灵隐寺', '浙江省图书馆', '0.0896475785341481'],
        ['杭州灵隐寺', '杭州晓书馆', '0.14226477204155247'],
        ['杭州灵隐寺', '浙江省博物馆', '0.37261821894154573'],
        ['杭州灵隐寺', '浙江省金华市东阳市横店镇', '1.1039170975869144']], dtype='<U32'),
 '浙江省博物馆': array([['浙江省博物馆', '浙江省图书馆', '0.2899342226322592'],
        ['浙江省博物馆', '杭州晓书馆', '0.3337550493652799'],
        ['浙江省博物馆', '杭州灵隐寺', '0.37261821894154573'],
        ['浙江省博物馆', '西湖景区', '0.39200247266004895'],
        ['浙江省博物馆', '西湖区蚂蚁金服', '0.3986826784442438'],
        ['浙江省博物馆', '杭州小河直街', '0.4141717826546668'],
        ['浙江省博物馆', '西湖青之坞', '0.4205975050396113'],
        ['浙江省博物馆', '浙江省金华市东阳市横店镇', '1.2164248631900676']], dtype='<U32'),
 '浙江省图书馆': array([['浙江省图书馆', '杭州灵隐寺', '0.0896475785341481'],
        ['浙江省图书馆', '西湖景区', '0.1039555645181722'],

In [5]:
def foo(l):
    index_ = []
    for i in range(len(l) - 1): 
        flag = 1
        for j in range(len(l) - i - 1):
            if l[j] > l[j + 1] and j != len(l):      
                l[j], l[j + 1] = l[j + 1], l[j]
                flag = 0
        if flag:
            break
    return l,index_

In [105]:
nodes

array([['杭州灵隐寺', '西湖景区', '0.026411843763575676'],
       ['杭州灵隐寺', '西湖区蚂蚁金服', '0.04023898467674691'],
       ['杭州灵隐寺', '西湖青之坞', '0.055969277373088766'],
       ['杭州灵隐寺', '杭州小河直街', '0.07522731741274191'],
       ['杭州灵隐寺', '浙江省图书馆', '0.0896475785341481'],
       ['杭州灵隐寺', '杭州晓书馆', '0.14226477204155247'],
       ['杭州灵隐寺', '浙江省博物馆', '0.37261821894154573'],
       ['杭州灵隐寺', '浙江省金华市东阳市横店镇', '1.1039170975869144'],
       ['浙江省博物馆', '浙江省图书馆', '0.2899342226322592'],
       ['浙江省博物馆', '杭州晓书馆', '0.3337550493652799'],
       ['浙江省博物馆', '杭州灵隐寺', '0.37261821894154573'],
       ['浙江省博物馆', '西湖景区', '0.39200247266004895'],
       ['浙江省博物馆', '西湖区蚂蚁金服', '0.3986826784442438'],
       ['浙江省博物馆', '杭州小河直街', '0.4141717826546668'],
       ['浙江省博物馆', '西湖青之坞', '0.4205975050396113'],
       ['浙江省博物馆', '浙江省金华市东阳市横店镇', '1.2164248631900676'],
       ['浙江省图书馆', '杭州灵隐寺', '0.0896475785341481'],
       ['浙江省图书馆', '西湖景区', '0.1039555645181722'],
       ['浙江省图书馆', '西湖区蚂蚁金服', '0.12558339008179156'],
       ['浙江省图书馆', '西湖青之坞'